In [ ]:
# default_exp core

# TREX Diviner Pipeline

> This package provides methods in combining LRO Diviner data with 1D thermal model results to derive partice size from measured cooling curves.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
def say_hello(to):
    "Say hello to somebody"
    return f"Hello {to}!"

In [ ]:
say_hello("Sylvain")

'Hello Sylvain!'

In [ ]:
assert say_hello("Jeremy") == "Hello Jeremy!"

In [ ]:
# export
class HelloSayer:
    "Say hello to `to` using `say_hello`"

    def __init__(self, to):
        self.to = to

    def say(self):
        "Do the saying"
        return say_hello(self.to)